In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
# !mkdir images

In [0]:
# !unzip -uq "/content/drive/My Drive/IDC_regular_ps50_idx5.zip" -d "/content/images"

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import matplotlib.pylab as plt
from glob import glob
import fnmatch
import cv2
from sklearn.model_selection import train_test_split
import random
from keras.preprocessing import image #for image preprocessing                  
from tqdm import tqdm # to maintain a progress bar
import os
import time
from datetime import timedelta
import math

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6419534045650282311
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8368405753411106872
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15915442457640380906
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15956161332
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8083513183164730532
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]






Using TensorFlow backend.


In [0]:
sess = tf.Session()
def init_variables():
    sess.run(tf.global_variables_initializer())
init_variables()

tf.__version__

'1.15.0'

In [0]:
# source: https://www.kaggle.com/paultimothymooney/predicting-idc-in-breast-cancer-histology-images/notebook
imagePatches = glob('/content/images/**/*.png', recursive=True) # search pathname/folder to find .png files recursively then parse files
for filename in imagePatches[0:10]:
    print(filename)

/content/images/13916/1/13916_idx5_x301_y1451_class1.png
/content/images/13916/1/13916_idx5_x1051_y951_class1.png
/content/images/13916/1/13916_idx5_x1801_y701_class1.png
/content/images/13916/1/13916_idx5_x1601_y551_class1.png
/content/images/13916/1/13916_idx5_x1601_y801_class1.png
/content/images/13916/1/13916_idx5_x601_y1251_class1.png
/content/images/13916/1/13916_idx5_x1851_y651_class1.png
/content/images/13916/1/13916_idx5_x951_y1451_class1.png
/content/images/13916/1/13916_idx5_x1051_y1301_class1.png
/content/images/13916/1/13916_idx5_x1651_y551_class1.png


In [0]:
patternZero = '*class0.png'
patternOne = '*class1.png'
classZero = fnmatch.filter(imagePatches, patternZero) #filename pattern matching returns subset of list of names
classOne = fnmatch.filter(imagePatches, patternOne)
print("IDC(-)\n\n",classZero[0:5],'\n')
print("IDC(+)\n\n",classOne[0:5])

IDC(-)

 ['/content/images/13916/0/13916_idx5_x1051_y1751_class0.png', '/content/images/13916/0/13916_idx5_x1101_y351_class0.png', '/content/images/13916/0/13916_idx5_x1251_y901_class0.png', '/content/images/13916/0/13916_idx5_x1451_y1551_class0.png', '/content/images/13916/0/13916_idx5_x1201_y751_class0.png'] 

IDC(+)

 ['/content/images/13916/1/13916_idx5_x301_y1451_class1.png', '/content/images/13916/1/13916_idx5_x1051_y951_class1.png', '/content/images/13916/1/13916_idx5_x1801_y701_class1.png', '/content/images/13916/1/13916_idx5_x1601_y551_class1.png', '/content/images/13916/1/13916_idx5_x1601_y801_class1.png']


In [0]:
def path_to_tensor(img_path):
    img = image.load_img(img_path, target_size=(227,227))
    x = image.img_to_array(img)
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [0]:
imageIndex = imagePatches[lowerIndex:upperIndex]
x_ = paths_to_tensor(imageIndex).astype('float32')/255

100%|██████████| 6000/6000 [00:07<00:00, 756.88it/s]


In [0]:
def proc_images(lowerIndex,upperIndex):
    """
    Returns two arrays: 
        x is an array of resized images
        y is an array of labels
    """ 
    x = []
    y = []
    for img in imagePatches[lowerIndex:upperIndex]:
        x.append(x_[lowerIndex:upperIndex])
        if img in classZero:
            y.append(0)
        elif img in classOne:
            y.append(1)
        else:
            return
    return x,y

In [0]:
%time
# n = len(imagePatches)
X,Y = proc_images(0,10000)
df = pd.DataFrame()
df["images"]=X
df["labels"]=Y
X2=df["images"]
Y2=df["labels"]
X2=np.array(X2)
imgs0=[]
imgs1=[]
imgs0 = X2[Y2==0] # (0 = Background, 1 = Number Plate)
imgs1 = X2[Y2==1] 

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 7.63 µs


In [0]:
from keras.utils import to_categorical
Y = to_categorical(Y)
print(Y)
print(Y.shape)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
(10000, 2)


In [0]:
(X_train, X_test, Y_train, Y_test) = train_test_split(X,Y, test_size=0.2)